# Code generation

In addition to being trained on massive volumes of text, many LLMs today were trained with billions of lines of code. LLMs such as `GPT-4o`, `Gemini 1.5 Flash`, and `Llama 32` can generate code, comment code, review code, optimize code, find bugs in code, and more, and they support dozens of programming languages. Let's try a few examples to see what `GPT-4o` can do. Start by using it to implement a bubble sort in Python:

In [ ]:
from openai import OpenAI
from IPython.display import Code, display

client = OpenAI(api_key='OPENAI_API_KEY')

prompt = '''
    Create a Python function that accepts an array of numbers as
    input, bubble sorts the numbers, and returns a sorted array.
    Return the source code only. Do not use markdown.
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0 # Use a low temperature for code generation
)

display(Code(response.choices[0].message.content, language='python'))

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# Example usage:
# sorted_array = bubble_sort([64, 34, 25, 12, 22, 11, 90])
# print(sorted_array)

LLMs are adept at explaining code, too, and even adding comments to source code. Suppose you wanted to write a tool to add comments to uncommented source-code files before checking them in. Here's a Python file that lacks comments:

In [2]:
with open('Data/create_database.py', 'r') as file:
    contents = file.read()
    display(Code(contents, language='python'))

import chromadb, uuid
from pypdf import PdfReader

client = chromadb.PersistentClient('chroma')
collection = client.create_collection(name='Electric_Vehicles')

file_names = [
    'electric_vehicles.pdf',
    'pev_consumer_handbook.pdf',
    'department-for-transport-ev-guide.pdf'
]

for file in file_names:
    print(f'Processing {file}')
    reader = PdfReader(file)

    for i, page in enumerate(reader.pages):
        text = page.extract_text()

        collection.add(
            documents=[text],
            metadatas=[{ 'file': file, 'page': i }],
            ids=[uuid.uuid4().hex]
        )

Now use `GPT-4o` to add comments:

In [3]:
with open('Data/create_database.py', 'r') as file:
    contents = file.read()

    prompt = f'''
        Add comments to the following source code. Return the source code only.
        Do not use markdown.

        {contents}
        '''

    messages = [{ 'role': 'user', 'content': prompt }]
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages
    )

    display(Code(response.choices[0].message.content, language='python'))

import chromadb, uuid
from pypdf import PdfReader

# Create a persistent client for ChromaDB a database
client = chromadb.PersistentClient('chroma')

# Create a new collection for storing electric vehicle documents
collection = client.create_collection(name='Electric_Vehicles')

# List of PDF files to process
file_names = [
    'electric_vehicles.pdf',
    'pev_consumer_handbook.pdf',
    'department-for-transport-ev-guide.pdf'
]

# Iterate over each file in the list
for file in file_names:
    print(f'Processing {file}')  # Print the name of the file being processed
    reader = PdfReader(file)  # Create a PDF reader object for the file

    # Iterate over each page in the PDF
    for i, page in enumerate(reader.pages):
        text = page.extract_text()  # Extract text from the page

        # Add the extracted text and metadata to the collection
        collection.add(
            documents=[text],  # Text extracted from the page
            metadatas=[{ 'file': file, 'page': i }],  # Metadata including file name and page number
            ids=[uuid.uuid4().hex]  # Unique ID for the document
        )

It works with C# source code as well. Here's an uncommented C# file that uses Microsoft's [ML.NET library](https://dotnet.microsoft.com/en-us/apps/machinelearning-ai/ml-dotnet) to train a machine-learning model that performs sentiment analysis:

In [4]:
with open('Data/Program.cs', 'r') as file:
    contents = file.read()
    display(Code(contents, language='C#'))

ï»¿using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Linq;

namespace SentimentAnalysis
{
    class Program
    {
        static readonly string _loadPath = "..\\..\\..\\Data\\yelp_labelled.tsv";
        static readonly string _savePath = "..\\..\\..\\Data\\Sentiment.zip";

        static void Main(string[] args)
        {
            var context = new MLContext(seed: 0);
            var data = context.Data.LoadFromTextFile<Input>(_loadPath, hasHeader: false);

            var trainTestData = context.Data.TrainTestSplit(data, testFraction: 0.2, seed: 0);
            var trainData = trainTestData.TrainSet;
            var testData = trainTestData.TestSet;

            var pipeline = context.Transforms.Text.FeaturizeText(outputColumnName: "Features", inputColumnName: "SentimentText")
                .Append(context.BinaryClassification.Trainers.FastTree(numberOfLeaves: 50, minimumExampleCountPerLeaf: 20));

            Console.WriteLine("Training the model...");
            var model = pipeline.Fit(trainData);

            var predictions = model.Transform(testData);
            var metrics = context.BinaryClassification.Evaluate(predictions, "Label");

            Console.WriteLine();
            Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");
            Console.WriteLine($"AUC: {metrics.AreaUnderPrecisionRecallCurve:P2}");
            Console.WriteLine($"F1: {metrics.F1Score:P2}");
            Console.WriteLine();

            Console.WriteLine("Saving the model...");
            context.Model.Save(model, data.Schema, _savePath);
        }
    }

    public class Input
    {
        [LoadColumn(0)]
        public string SentimentText;

        [LoadColumn(1), ColumnName("Label")]
        public bool Sentiment;
    }

    public class Output
    {
        [ColumnName("PredictedLabel")]
        public bool Prediction { get; set; }
        public float Probability { get; set; }
    }
}

And here it is after `GPT-4o` dresses it up:

In [5]:
with open('Data/Program.cs', 'r') as file:
    contents = file.read()

    prompt = f'''
        Add comments to the following source code. Return the source code only.
        Do not use markdown.

        {contents}
        '''

    messages = [{ 'role': 'user', 'content': prompt }]
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages
    )

    display(Code(response.choices[0].message.content, language='C#'))

using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Linq;

namespace SentimentAnalysis
{
    class Program
    {
        // Path to the input data file.
        static readonly string _loadPath = "..\\..\\..\\Data\\yelp_labelled.tsv";
        // Path where the model will be saved.
        static readonly string _savePath = "..\\..\\..\\Data\\Sentiment.zip";

        static void Main(string[] args)
        {
            // Initialize a new MLContext.
            var context = new MLContext(seed: 0);
            
            // Load data from the TSV file into an IDataView.
            var data = context.Data.LoadFromTextFile<Input>(_loadPath, hasHeader: false);

            // Split the data into training and test datasets.
            var trainTestData = context.Data.TrainTestSplit(data, testFraction: 0.2, seed: 0);
            var trainData = trainTestData.TrainSet;
            var testData = trainTestData.TestSet;

            // Define the data preparation and model training pipeline.
            var pipeline = context.Transforms.Text.FeaturizeText(outputColumnName: "Features", inputColumnName: "SentimentText")
                .Append(context.BinaryClassification.Trainers.FastTree(numberOfLeaves: 50, minimumExampleCountPerLeaf: 20));

            Console.WriteLine("Training the model...");
            // Train the model using the training dataset.
            var model = pipeline.Fit(trainData);

            // Make predictions on the test dataset.
            var predictions = model.Transform(testData);
            // Evaluate the model's performance.
            var metrics = context.BinaryClassification.Evaluate(predictions, "Label");

            // Output the model evaluation metrics.
            Console.WriteLine();
            Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");
            Console.WriteLine($"AUC: {metrics.AreaUnderPrecisionRecallCurve:P2}");
            Console.WriteLine($"F1: {metrics.F1Score:P2}");
            Console.WriteLine();

            Console.WriteLine("Saving the model...");
            // Save the trained model to a zip file.
            context.Model.Save(model, data.Schema, _savePath);
        }
    }

    public class Input
    {
        // Represents the first column in the dataset as a string.
        [LoadColumn(0)]
        public string SentimentText;

        // Represents the second column in the dataset as a boolean label.
        [LoadColumn(1), ColumnName("Label")]
        public bool Sentiment;
    }

    public class Output
    {
        // Represents the predicted label output of the model.
        [ColumnName("PredictedLabel")]
        public bool Prediction { get; set; }
        // Represents the probability of the prediction.
        public float Probability { get; set; }
    }
}

Speaking of C#: Can `GPT-4o` rewrite a block of C# code that manually iterates over a `List` to use LINQ?

In [6]:
prompt = '''
    Rewrite the following code to use LINQ. Return the source code only.
    Do not use markdown.

    var picks = new List<DailyStock>();

    foreach (var stock in stocks)
    {
        if (stock.Close > stock.Open)
            picks.Add(stock);
    }

    foreach(var pick in picks)
    {
        Console.WriteLine($"{pick.Symbol}: {pick.Open:c} -> {pick.Close:c}");
    }
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0
)

display(Code(response.choices[0].message.content, language='C#'))

var picks = stocks.Where(stock => stock.Close > stock.Open).ToList();
picks.ForEach(pick => Console.WriteLine($"{pick.Symbol}: {pick.Open:c} -> {pick.Close:c}"));

One of the more remarkable aspects of LLMs is their ability to find bugs in code. Just after my book went to the printer in 2022, I discovered a bug in the latest version of Scikit-learn that prevented some of my samples from working properly. I spent a couple of hours in the source code and found the bug. I filed a [bug report](https://github.com/scikit-learn/scikit-learn/issues/24942) and the Scikit team confirmed the bug and promised to fix it in the next version. I had to scramble to rewrite some of the code samples in my book to work around the bug and get the changes to the printer before the presses started rolling. Let's see if `GPT-4o` can find the bug:

In [7]:
with open('Data/lfw.py', 'r') as file:
    contents = file.read()

    prompt = f'''
        Find the bug in the code below that prevents the _load_imgs function
        from properly cropping images as specified by the slice_ parameter:

        {contents}
        '''

    messages = [{ 'role': 'user', 'content' : prompt }]
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages
    )
    
    print(response.choices[0].message.content)

The issue in the code is with how the `crop` method is being used. The `crop` method is called on `pil_img` but the result is not being reassigned or used further. The `Image.crop` method in PIL does not modify the original image in place; instead, it returns a new `Image` object representing the cropped area.

To fix this bug, you need to capture the return value of the `crop` method so that you use the cropped version of the image for further processing. Here's the corrected part of the code:

```python
# Incorrect usage:
# pil_img.crop((w_slice.start, h_slice.start, w_slice.stop, h_slice.stop))

# Corrected usage:
pil_img = pil_img.crop((w_slice.start, h_slice.start, w_slice.stop, h_slice.stop))
```

With this change, the cropped image will be stored in `pil_img`, and any subsequent operations, such as resizing and conversion to a numpy array, will be performed on this cropped image.


The fact that foundational LLMs are capable of understanding code and generating code of their own is a game-changer for application developers. Tools such as [GitHub Copilot](https://github.com/features/copilot) are making programmers more productive and revolutionizing the way software is written. Without LLMs, such tools wouldn't be possible.